# Field Analysis


In [1]:
import pandas as pd

In [2]:
pd.set_option("display.max_columns", None)

## Load Data

First extract the raw data from a `.pkl` file. The reason for storing the fields in a `.pkl` file is to ensure that data structures such as lists do not get converted into strings.


In [3]:
df = pd.read_pickle("../fields.pkl")
df


,hasResults,protocolSection.identificationModule.nctId,protocolSection.identificationModule.orgStudyIdInfo.id,protocolSection.identificationModule.secondaryIdInfos,protocolSection.identificationModule.organization.fullName,protocolSection.identificationModule.organization.class,protocolSection.identificationModule.briefTitle,protocolSection.identificationModule.officialTitle,protocolSection.statusModule.statusVerifiedDate,protocolSection.statusModule.overallStatus,protocolSection.statusModule.expandedAccessInfo.hasExpandedAccess,protocolSection.statusModule.startDateStruct.date,protocolSection.statusModule.primaryCompletionDateStruct.date,protocolSection.statusModule.primaryCompletionDateStruct.type,protocolSection.statusModule.completionDateStruct.date,protocolSection.statusModule.completionDateStruct.type,protocolSection.statusModule.studyFirstSubmitDate,protocolSection.statusModule.studyFirstSubmitQcDate,protocolSection.statusModule.studyFirstPostDateStruct.date,protocolSection.statusModule.studyFirstPostDateStruct.type,protocolSection.statusModule.lastUpdateSubmitDate,protocolSection.statusModule.lastUpdatePostDateStruct.date,protocolSection.statusModule.lastUpdatePostDateStruct.type,protocolSection.sponsorCollaboratorsModule.responsibleParty.type,protocolSection.sponsorCollaboratorsModule.leadSponsor.name,protocolSection.sponsorCollaboratorsModule.leadSponsor.class,protocolSection.sponsorCollaboratorsModule.collaborators,protocolSection.descriptionModule.briefSummary,protocolSection.descriptionModule.detailedDescription,protocolSection.conditionsModule.conditions,protocolSection.conditionsModule.keywords,protocolSection.designModule.studyType,protocolSection.designModule.phases,protocolSection.designModule.designInfo.primaryPurpose,protocolSection.designModule.designInfo.maskingInfo.masking,protocolSection.armsInterventionsModule.interventions,protocolSection.outcomesModule.primaryOutcomes,protocolSection.outcomesModule.secondaryOutcomes,protocolSection.eligibilityModule.eligibilityCriteria,protocolSection.eligibilityModule.healthyVolunteers,protocolSection.eligibilityModule.sex,protocolSection.eligibilityModule.minimumAge,protocolSection.eligibilityModule.maximumAge,protocolSection.eligibilityModule.stdAges,protocolSection.contactsLocationsModule.overallOfficials,protocolSection.contactsLocationsModule.locations,derivedSection.miscInfoModule.versionHolder,derivedSection.conditionBrowseModule.meshes,derivedSection.conditionBrowseModule.ancestors,derivedSection.conditionBrowseModule.browseLeaves,derivedSection.conditionBrowseModule.browseBranches,derivedSection.interventionBrowseModule.meshes,derivedSection.interventionBrowseModule.ancestors,derivedSection.interventionBrowseModule.browseLeaves,derivedSection.interventionBrowseModule.browseBranches,protocolSection.designModule.designInfo.allocation,protocolSection.designModule.designInfo.interventionModel,protocolSection.designModule.enrollmentInfo.count,protocolSection.designModule.enrollmentInfo.type,protocolSection.oversightModule.oversightHasDmc,protocolSection.armsInterventionsModule.armGroups,protocolSection.referencesModule.references,protocolSection.statusModule.startDateStruct.type,protocolSection.oversightModule.isFdaRegulatedDrug,protocolSection.oversightModule.isFdaRegulatedDevice,protocolSection.designModule.designInfo.interventionModelDescription,protocolSection.designModule.designInfo.maskingInfo.whoMasked,protocolSection.ipdSharingStatementModule.ipdSharing,protocolSection.statusModule.lastKnownStatus,protocolSection.designModule.designInfo.maskingInfo.maskingDescription,protocolSection.identificationModule.acronym,protocolSection.statusModule.resultsFirstSubmitDate,protocolSection.statusModule.resultsFirstSubmitQcDate,protocolSection.statusModule.resultsFirstPostDateStruct.date,protocolSection.statusModule.resultsFirstPostDateStruct.type,resultsSection.participantFlowModule.preAssignmentDetails,resultsSection.participantFlowModule.recruitmentDetails,results

## Complex Columns


In [4]:
def is_complex_column(df: pd.DataFrame) -> bool:
    if isinstance(df, list):
        for cell in df:
            if isinstance(cell, dict):
                return True
    return False


complex_columns = []
for col in df.columns:
    if df[col].apply(is_complex_column).any():
        complex_columns.append(col)

complex_columns


['protocolSection.identificationModule.secondaryIdInfos',
 'protocolSection.sponsorCollaboratorsModule.collaborators',
 'protocolSection.armsInterventionsModule.interventions',
 'protocolSection.outcomesModule.primaryOutcomes',
 'protocolSection.outcomesModule.secondaryOutcomes',
 'protocolSection.contactsLocationsModule.overallOfficials',
 'protocolSection.contactsLocationsModule.locations',
 'derivedSection.conditionBrowseModule.meshes',
 'derivedSection.conditionBrowseModule.ancestors',
 'derivedSection.conditionBrowseModule.browseLeaves',
 'derivedSection.conditionBrowseModule.browseBranches',
 'derivedSection.interventionBrowseModule.meshes',
 'derivedSection.interventionBrowseModule.ancestors',
 'derivedSection.interventionBrowseModule.browseLeaves',
 'derivedSection.interventionBrowseModule.browseBranches',
 'protocolSection.armsInterventionsModule.armGroups',
 'protocolSection.referencesModule.references',
 'resultsSection.participantFlowModule.groups',
 'resultsSection.partici

In [5]:
complex_fields = df[complex_columns]
complex_fields


,protocolSection.identificationModule.secondaryIdInfos,protocolSection.sponsorCollaboratorsModule.collaborators,protocolSection.armsInterventionsModule.interventions,protocolSection.outcomesModule.primaryOutcomes,protocolSection.outcomesModule.secondaryOutcomes,protocolSection.contactsLocationsModule.overallOfficials,protocolSection.contactsLocationsModule.locations,derivedSection.conditionBrowseModule.meshes,derivedSection.conditionBrowseModule.ancestors,derivedSection.conditionBrowseModule.browseLeaves,derivedSection.conditionBrowseModule.browseBranches,derivedSection.interventionBrowseModule.meshes,derivedSection.interventionBrowseModule.ancestors,derivedSection.interventionBrowseModule.browseLeaves,derivedSection.interventionBrowseModule.browseBranches,protocolSection.armsInterventionsModule.armGroups,protocolSection.referencesModule.references,resultsSection.participantFlowModule.groups,resultsSection.participantFlowModule.periods,resultsSection.baselineCharacteristicsModule.groups,resultsSection.baselineCharacteristicsModule.denoms,resultsSection.baselineCharacteristicsModule.measures,resultsSection.outcomeMeasuresModule.outcomeMeasures,resultsSection.adverseEventsModule.eventGroups,resultsSection.adverseEventsModule.seriousEvents,resultsSection.adverseEventsModule.otherEvents,documentSection.largeDocumentModule.largeDocs,protocolSection.contactsLocationsModule.centralContacts,protocolSection.referencesModule.seeAlsoLinks,protocolSection.outcomesModule.otherOutcomes,annotationSection.annotationModule.unpostedAnnotation.unpostedEvents,derivedSection.miscInfoModule.submissionTracking.submissionInfos,protocolSection.referencesModule.availIpds,annotationSection.annotationModule.violationAnnotation.violationEvents
0,"[{'id': 'CDR0000340983', 'type': 'REGISTRY', '...","[{'name': 'National Cancer Institute (NCI)', '...","[{'type': 'BIOLOGICAL', 'name': 'sargramostim'}]",[{'measure': 'Cytogenetic response (complete a...,[{'measure': 'Toxicity as assessed by the Expa...,"[{'name': 'Istvan Molnar, MD', 'affiliation': ...","[{'facility': 'CCOP - Western Regional, Arizon...","[{'id': 'D000007938', 'term': 'Leukemia'}, {'i...","[{'id': 'D000009370', 'term': 'Neoplasms by Hi...","[{'id': 'M10945', 'name': 'Leukemia', 'asFound...","[{'abbrev': 'BC04', 'name': 'Neoplasms'}, {'ab...","[{'id': 'C000081222', 'term': 'Sargramostim'}]","[{'id': 'D000007155', 'term': 'Immunologic Fac...","[{'id': 'M257633', 'name': 'Molgramostim', 're...","[{'abbrev': 'ANeo', 'name': 'Antineoplastic Ag...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[{'id': 'HARECCTR0500057'}],NaN,"[{'type': 'DRUG', 'name': 'Vardenafil 10mg'}]",[{'measure': 'Mean maximal change of the stand...,[{'measure': 'Mean maximal post-baseline chang...,"[{'name': 'Chi Fai Ng, Dr', 'affiliation': 'De...","[{'facility': 'Prince of Wales Hospital', 'cit...","[{'id': 'D000007172', 'term': 'Erectile Dysfun...","[{'id': 'D000010335', 'term': 'Pathologic Proc...","[{'id': 'M10217', 'name': 'Erectile Dysfunctio...","[{'abbrev': 'BXS', 'name': 'Urinary Tract, Sex...","[{'id': 'D000069058', 'term': 'Vardenafil Dihy...","[{'id': 'D000014665', 'term': 'Vasodilator Age...","[{'id': 'M19586', 'name': 'Doxazosin', 'releva...","[{'abbrev': 'AnAg', 'name': 'Antihypertensive ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"[{'type': 'OTHER', 'name': 'Distillation based...",[{'measure': 'To demonstrate a reduction in th...,[{'measure': 'To explore changes from baseline...,"[{'name': 'Katarzyna Jarus-Dziedzic, MD', 'aff...","[{'facility': 'MTZ Clinical Research Inc.', 'c...",NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 'M12478', 'name': 'Nicotine', 'relevan...","[{'abbrev': 'All', 'name': 'All Drugs and Chem...","[{'label': '1. SMAR', 'type': 'EXPERIMENTAL', ...","[{'pmid': '26817490', 'type': 'RESULT', 'citat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'id': 'IRB00207577', 'type': 'OTHER', 'domai...","[{'nam

In [6]:
df.drop(complex_columns, axis=1)
df


,hasResults,protocolSection.identificationModule.nctId,protocolSection.identificationModule.orgStudyIdInfo.id,protocolSection.identificationModule.secondaryIdInfos,protocolSection.identificationModule.organization.fullName,protocolSection.identificationModule.organization.class,protocolSection.identificationModule.briefTitle,protocolSection.identificationModule.officialTitle,protocolSection.statusModule.statusVerifiedDate,protocolSection.statusModule.overallStatus,protocolSection.statusModule.expandedAccessInfo.hasExpandedAccess,protocolSection.statusModule.startDateStruct.date,protocolSection.statusModule.primaryCompletionDateStruct.date,protocolSection.statusModule.primaryCompletionDateStruct.type,protocolSection.statusModule.completionDateStruct.date,protocolSection.statusModule.completionDateStruct.type,protocolSection.statusModule.studyFirstSubmitDate,protocolSection.statusModule.studyFirstSubmitQcDate,protocolSection.statusModule.studyFirstPostDateStruct.date,protocolSection.statusModule.studyFirstPostDateStruct.type,protocolSection.statusModule.lastUpdateSubmitDate,protocolSection.statusModule.lastUpdatePostDateStruct.date,protocolSection.statusModule.lastUpdatePostDateStruct.type,protocolSection.sponsorCollaboratorsModule.responsibleParty.type,protocolSection.sponsorCollaboratorsModule.leadSponsor.name,protocolSection.sponsorCollaboratorsModule.leadSponsor.class,protocolSection.sponsorCollaboratorsModule.collaborators,protocolSection.descriptionModule.briefSummary,protocolSection.descriptionModule.detailedDescription,protocolSection.conditionsModule.conditions,protocolSection.conditionsModule.keywords,protocolSection.designModule.studyType,protocolSection.designModule.phases,protocolSection.designModule.designInfo.primaryPurpose,protocolSection.designModule.designInfo.maskingInfo.masking,protocolSection.armsInterventionsModule.interventions,protocolSection.outcomesModule.primaryOutcomes,protocolSection.outcomesModule.secondaryOutcomes,protocolSection.eligibilityModule.eligibilityCriteria,protocolSection.eligibilityModule.healthyVolunteers,protocolSection.eligibilityModule.sex,protocolSection.eligibilityModule.minimumAge,protocolSection.eligibilityModule.maximumAge,protocolSection.eligibilityModule.stdAges,protocolSection.contactsLocationsModule.overallOfficials,protocolSection.contactsLocationsModule.locations,derivedSection.miscInfoModule.versionHolder,derivedSection.conditionBrowseModule.meshes,derivedSection.conditionBrowseModule.ancestors,derivedSection.conditionBrowseModule.browseLeaves,derivedSection.conditionBrowseModule.browseBranches,derivedSection.interventionBrowseModule.meshes,derivedSection.interventionBrowseModule.ancestors,derivedSection.interventionBrowseModule.browseLeaves,derivedSection.interventionBrowseModule.browseBranches,protocolSection.designModule.designInfo.allocation,protocolSection.designModule.designInfo.interventionModel,protocolSection.designModule.enrollmentInfo.count,protocolSection.designModule.enrollmentInfo.type,protocolSection.oversightModule.oversightHasDmc,protocolSection.armsInterventionsModule.armGroups,protocolSection.referencesModule.references,protocolSection.statusModule.startDateStruct.type,protocolSection.oversightModule.isFdaRegulatedDrug,protocolSection.oversightModule.isFdaRegulatedDevice,protocolSection.designModule.designInfo.interventionModelDescription,protocolSection.designModule.designInfo.maskingInfo.whoMasked,protocolSection.ipdSharingStatementModule.ipdSharing,protocolSection.statusModule.lastKnownStatus,protocolSection.designModule.designInfo.maskingInfo.maskingDescription,protocolSection.identificationModule.acronym,protocolSection.statusModule.resultsFirstSubmitDate,protocolSection.statusModule.resultsFirstSubmitQcDate,protocolSection.statusModule.resultsFirstPostDateStruct.date,protocolSection.statusModule.resultsFirstPostDateStruct.type,resultsSection.participantFlowModule.preAssignmentDetails,resultsSection.participantFlowModule.recruitmentDetails,results

In [7]:
normalized_df = pd.DataFrame()

for index, row in complex_fields.iterrows():
    for col in complex_columns:
        if isinstance(row[col], list):
            expanded = pd.json_normalize(row[col])
            expanded.columns = [f"{col}.{c}" for c in expanded.columns]
            normalized_df = pd.concat([normalized_df, expanded], ignore_index=True)

normalized_df


KeyboardInterrupt: 

## Summary Statistics


In [ ]:
df.describe(include="all")